# Creación de las firmas de genes requeridas por muchos GSEA de tipo FCS
```
Autor: Martín Sende Pombo
Fecha de creación: 31-12-2023
Fecha de última modificación: 14-01-2024
```

Para efectuar un análisis de enriquecimiento de conjuntos de genes ([GSEA](https://en.wikipedia.org/wiki/Gene_set_enrichment_analysis)) o análisis de enriquecimiento funcional, empleando un método tipo [FCS](https://doi.org/10.3389/fgene.2020.00654) (puntuación de la clase funcional) como son [blitzGSEA](https://doi.org/10.1093/bioinformatics/btac076) y [CERNO](https://doi.org/10.1093/bioinformatics/btz447), se requiere una biblioteca de conjuntos de genes y una firma de expresión génica (otros métodos FCS parten de un conjunto de datos de expresión).

La función de este cuardeno de Jupyter es generar dichas firmas de genes en [formato .rnk](https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#Ranked_Gene_Lists), para poder utilizarlas como datos de entrada para las herramientas mencionadas, a partir de los libros de Excel obtenidos tras ejecutar cualquiera de los flujos de trabajo contenidos en la carpeta "Example_workflows" de la SL-Cloud-es.

In [1]:
# Posiblemente ya tenga esta biblioteca instalda por defecto, pero si no es el caso, ejecute este bloque de código.
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable


## Generación de los archivos .rnk temporales
Los siguientes bloques de código generan archivos .rnk incompletos, pues les falta una fila correspondiente al gen para el que se buscaron los pares SL que contiene cada uno. Aún así, son necesarios y además puede utilizarlos como firmas de expresión génica para el GSEA, siempre que no quiera incluir en el análisis los genes de búsqueda. 

Dichos archivos son almacenados en el subdirectorio denominado "Provisionales" de esta misma carpeta y, posteriormente, podrá copiarlos y/o eliminarlos según conveniencia. No obstante, tenga en cuenta que si corre este código más de una vez seguida (sin ejecutar también los siguientes pasos), para leer hojas de cálculo creadas a partir de listas de genes con elementos repetidos, los archivos de lista ordenada (RNK) derivados de esas coincidencias se sobreescribirán con el más reciente.

### Para tablas de 5 columnas con la siguiente disposición:
* Columna A -> **Inactive**: El gen de entrada del usuario.

* Columna B -> **SL_Candidate/SDL_Candidate**: El par "SL" o "SDL" candidato. 

* Columna C -> **PValue0 or FDR0**: El valor p agregado o FDR obtenido con el método de Fisher, para PanCanAtlas o CRISPR. 

* Columna D -> **PValue1 or FDR1**: El valor p agregado o FDR obtenido con el método de Fisher, para CCLE o ARNhc.

* Columna E -> **Tissue**: El/los tejido/s en el/los que se ha realizado el análisis.

Si se modifica el nombre de las columnas en el código siguiente, estas pueden encontrarse en otro orden en la tabla, a excepción de la columna B (siempre debe ser la 2ª empezando por la izquierda).

In [38]:
import os
import pandas as pd

def procesar_tabla(file_path):
    # Leer el archivo .xlsx y almacenar la tabla en una variable
    df = pd.read_excel(file_path)

    # Obtener los nombres de las columnas
    columna_E = df.columns[-1]
    columna_C = df.columns[2]
    columna_D = df.columns[3]
    columna_A = df.columns[0]

    # Eliminar la columna especificada
    if columna_E in df.columns:
        df = df.drop(columns=[columna_E])

    # Procesar las columnas según las condiciones dadas
    df[columna_D] = df.apply(lambda row: pd.NA if pd.isna(row[columna_D]) or (pd.notna(row[columna_C]) and row[columna_C] <= row[columna_D]) else row[columna_D], axis=1)
    df[columna_C] = df.apply(lambda row: row[columna_D] if pd.isna(row[columna_C]) or (pd.notna(row[columna_D]) and row[columna_C] > row[columna_D]) else row[columna_C], axis=1)

    # Eliminar la columna especificada
    if columna_D in df.columns:
        df = df.drop(columns=[columna_D])

    # Crear una carpeta para los archivos .rnk si no existe
    output_folder = 'Provisionales'
    os.makedirs(output_folder, exist_ok=True)

    # Guardar las filas en un archivo .rnk en la carpeta
    grouped_by_inactive = df.groupby(columna_A)
    for inactive_char, group in grouped_by_inactive:
        output_file_path = os.path.join(output_folder, f'{inactive_char}.rnk')
        if len(group) > 1:
            group.drop(columns=[columna_A]).to_csv(output_file_path, sep='\t', header=False, index=False)
        else:
            group.drop(columns=[columna_A]).to_csv(output_file_path, sep='\t', header=False, index=False)

# Uso del código con un archivo específico
archivo_a_procesar = 'Listas_candidatos_SLoSDL/CEGP_SLoSDL_resultados.xlsx'
procesar_tabla(archivo_a_procesar)

print("Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.")

Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.


#### Alternativa para tablas de 5 columnas
Claro, puedo modificar el código para que sea más general y funcione con cualquier tabla que tenga una estructura similar. Aquí está el código modificado:

In [4]:
import os
import pandas as pd

def procesar_tabla(file_path):
    # Leer el archivo .xlsx y almacenar la tabla en una variable
    df = pd.read_excel(file_path)

    # Eliminar la columna especificada (por ejemplo, "Tissue")
    columna_eliminar = 'Tejido'
    if columna_eliminar in df.columns:
        df = df.drop(columns=[columna_eliminar])

    # Procesar las columnas según las condiciones dadas
    columna_3 = 'FDR0'
    columna_4 = 'FDR1'
    df[columna_4] = df.apply(lambda row: pd.NA if pd.isna(row[columna_4]) or (pd.notna(row[columna_3]) and row[columna_3] <= row[columna_4]) else row[columna_4], axis=1)
    df[columna_3] = df.apply(lambda row: row[columna_4] if pd.isna(row[columna_3]) or (pd.notna(row[columna_4]) and row[columna_3] > row[columna_4]) else row[columna_3], axis=1)

    # Eliminar la columna especificada (por ejemplo, "FDR1")
    columna_eliminar = 'FDR1'
    if columna_eliminar in df.columns:
        df = df.drop(columns=[columna_eliminar])

    # Crear una carpeta para los archivos .rnk si no existe
    output_folder = 'Provisionales'
    os.makedirs(output_folder, exist_ok=True)

    # Guardar las filas en un archivo .rnk en la carpeta
    columna_inactive = 'Inactivo'
    grouped_by_inactive = df.groupby(columna_inactive)
    for inactive_char, group in grouped_by_inactive:
        output_file_path = os.path.join(output_folder, f'{inactive_char}.rnk')
        if len(group) > 1:
            group.drop(columns=[columna_inactive]).to_csv(output_file_path, sep='\t', header=False, index=False)
        else:
            group.drop(columns=[columna_inactive]).to_csv(output_file_path, sep='\t', header=False, index=False)

# Uso del código con un archivo específico
archivo_a_procesar = 'Listas_candidatos_SLoSDL/Cinco_columnas_ejemplo.xlsx'
procesar_tabla(archivo_a_procesar)

print("Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.")

Proceso completado. Archivos .rnk temporales guardados en la carpeta 'Provisionales'.


### Para tablas de 4 columnas con la siguiente disposición:
* Columna A -> **Inactive**: El gen de entrada del usuario.

* Columna B -> **SL_Candidate/SDL_Candidate**: El par "SL" o "SDL" candidato. 

* Columna C -> **PValue0 or FDR0** o **PValue1 or FDR1**: El valor p agregado o FDR obtenido con el método de Fisher, para una base de datos o método de cribado funcional determinado. 

* Columna D -> **Tissue**: El/los tejido/s en el/los que se ha realizado el análisis.

Si se modifica el nombre de las columnas en el código siguiente, estas pueden encontrarse en otro orden en la tabla, a excepción de la columna B (siempre debe ser la 2ª empezando por la izquierda).

In [24]:
import os
import pandas as pd

def procesar_tabla(file_path):
    # Leer el archivo .xlsx y almacenar la tabla en una variable
    df = pd.read_excel(file_path)

    # Obtener los nombres de las columnas
    columna_D = df.columns[-1]
    columna_A = df.columns[0]

    # Eliminar la columna especificada
    if columna_D in df.columns:
        df = df.drop(columns=[columna_D])

    # Crear una carpeta para los archivos .rnk si no existe
    output_folder = 'Provisionales'
    os.makedirs(output_folder, exist_ok=True)

    # Guardar las filas en un archivo .rnk en la carpeta
    grouped_by_inactive = df.groupby(columna_A)
    for inactive_char, group in grouped_by_inactive:
        output_file_path = os.path.join(output_folder, f'{inactive_char}.rnk')
        if len(group) > 1:
            group.drop(columns=[columna_A]).to_csv(output_file_path, sep='\t', header=False, index=False)
        else:
            group.drop(columns=[columna_A]).to_csv(output_file_path, sep='\t', header=False, index=False)

# Uso del código con un archivo específico
archivo_a_procesar = 'Listas_candidatos_SLoSDL/FunEx_SLoSDL_resultados.xlsx'
procesar_tabla(archivo_a_procesar)

print("Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.")

Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.


#### Alternativa para tablas de 4 columnas
Claro, puedo modificar el código para que sea más general y funcione con cualquier tabla que tenga una estructura similar. Aquí está el código modificado:

In [31]:
import os
import pandas as pd

def procesar_tabla(file_path):
    # Leer el archivo .xlsx y almacenar la tabla en una variable
    df = pd.read_excel(file_path)

    # Eliminar la columna especificada (por ejemplo, "Tissue")
    columna_eliminar = 'Tissue'
    if columna_eliminar in df.columns:
        df = df.drop(columns=[columna_eliminar])

    # Crear una carpeta para los archivos .rnk si no existe
    output_folder = 'Provisionales'
    os.makedirs(output_folder, exist_ok=True)

    # Guardar las filas en un archivo .rnk en la carpeta
    columna_inactive = 'Inactive'
    grouped_by_inactive = df.groupby(columna_inactive)
    for inactive_char, group in grouped_by_inactive:
        output_file_path = os.path.join(output_folder, f'{inactive_char}.rnk')
        if len(group) > 1:
            group.drop(columns=[columna_inactive]).to_csv(output_file_path, sep='\t', header=False, index=False)
        else:
            group.drop(columns=[columna_inactive]).to_csv(output_file_path, sep='\t', header=False, index=False)

# Uso del código con un archivo específico
archivo_a_procesar = 'Listas_candidatos_SLoSDL/SoF_SLoSDL_resultados.xlsx'
procesar_tabla(archivo_a_procesar)

print("Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.")

Proceso completado. Archivos .rnk incompletos guardados en la carpeta 'Provisionales'.


## Modificación de los archivos RNK temporales para ontener los definitivos
Si los archivos .rnk de partida se encuentran en otro directorio, simplemente necesitas proporcionar la ruta de ese directorio en lugar de asumir que están en el directorio actual. Modificaré el código para que puedas especificar la ruta del directorio que contiene los archivos .rnk. Aquí está la versión actualizada:

In [39]:
import os

# Ruta del directorio que contiene los archivos .rnk
directorio_archivos = 'Provisionales'

# Crear la carpeta si no existe
definativas_folder = 'Definitivas'
if not os.path.exists(definativas_folder):
    os.makedirs(definativas_folder)

# Obtener la lista de archivos .rnk en el directorio especificado
archivos_rnk = [archivo for archivo in os.listdir(directorio_archivos) if archivo.endswith('.rnk')]

# Función para realizar las modificaciones en cada archivo .rnk
def modificar_archivo(rnk_filename):
    ruta_completa = os.path.join(directorio_archivos, rnk_filename)
    
    with open(ruta_completa, 'r') as archivo:
        lines = archivo.readlines()

    # Obtener los valores de la segunda columna
    valores_segunda_columna = [float(line.split('\t')[1]) for line in lines]

    # Encontrar el índice del valor mínimo
    indice_valor_minimo = valores_segunda_columna.index(min(valores_segunda_columna))

    # Obtener el nombre del archivo sin la extensión
    nombre_archivo = os.path.splitext(rnk_filename)[0]

    # Realizar las modificaciones según las condiciones
    if indice_valor_minimo == 0:
        nueva_fila = f"{nombre_archivo}\t0\n"
        lines.insert(0, nueva_fila)
    elif indice_valor_minimo == len(valores_segunda_columna) - 1:
        nueva_fila = f"{nombre_archivo}\t0\n"
        lines.append(nueva_fila)
    else:
        nueva_fila = f"{nombre_archivo}\t0\n"
        lines.insert(0, nueva_fila)

    # Guardar el archivo modificado en la carpeta Definitivas
    nuevo_rnk_filename = os.path.join(definativas_folder, rnk_filename)
    with open(nuevo_rnk_filename, 'w') as nuevo_archivo:
        nuevo_archivo.writelines(lines)

# Aplicar la modificación a cada archivo .rnk
for archivo_rnk in archivos_rnk:
    modificar_archivo(archivo_rnk)

print("Proceso completado. Firmas de genes definitivas para GSEA guardadas en la carpeta 'Definitivas'.")

Proceso completado. Firmas de genes definitivas para GSEA guardadas en la carpeta 'Definitivas'.


## Comprobación de la ausencia de valores de clasificación repetidos en los archivos RNK

In [40]:
import os
from collections import defaultdict
import shutil

def verificar_archivos_rnk(carpeta):
    carpeta_origen = "Definitivas"
    carpeta_destino = "Valores_duplicados"
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    archivos_rnk = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith(".rnk")]

    for archivo in archivos_rnk:
        ruta_archivo = os.path.join(carpeta_origen, archivo)
        genes_repetidos = encontrar_genes_repetidos(ruta_archivo)

        if genes_repetidos:
            print(f"Archivo afectado: {archivo}")
            print("Advertencia: Esta lista de genes ordenados contiene valores de clasificación duplicados. Esto puede dar lugar a una ordenación arbitraria de los genes y a resultados erróneos, por lo que debe corregirse antes del GSEA.")
            print("El bloque de código que encontrará a continuación, le permitirá solventar este problema.")

            # Mover el archivo a la carpeta "Valores_duplicados"
            destino_archivo = os.path.join(carpeta_destino, archivo)
            shutil.move(ruta_archivo, destino_archivo)

            print(f"El archivo {archivo} ha sido movido a la carpeta 'Valores_duplicados'.")
        else:
            print(f"No se encontraron valores de clasificación duplicados en {archivo}. Por tanto, si lo desea puede realizar el GSEA para esta firma de genes.")

def encontrar_genes_repetidos(ruta_archivo):
    genes_contador = defaultdict(int)

    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            _, valor = linea.strip().split('\t')
            genes_contador[valor] += 1

    genes_repetidos = [gen for gen, count in genes_contador.items() if count > 1]
    return genes_repetidos

# Cambia la ruta de la carpeta según donde tengas tus archivos .rnk
carpeta_contenedora = 'Definitivas'
verificar_archivos_rnk(carpeta_contenedora)

print("Comprobación finalizada.")

Archivo afectado: AFMID.rnk
Advertencia: Esta lista de genes ordenados contiene valores de clasificación duplicados. Esto puede dar lugar a una ordenación arbitraria de los genes y a resultados erróneos, por lo que debe corregirse antes del GSEA.
El bloque de código que encontrará a continuación, le permitirá solventar este problema.
El archivo AFMID.rnk ha sido movido a la carpeta 'Valores_duplicados'.
No se encontraron valores de clasificación duplicados en ASPDH.rnk. Por tanto, si lo desea puede realizar el GSEA para esta firma de genes.
No se encontraron valores de clasificación duplicados en HAAO.rnk. Por tanto, si lo desea puede realizar el GSEA para esta firma de genes.
No se encontraron valores de clasificación duplicados en IDO1.rnk. Por tanto, si lo desea puede realizar el GSEA para esta firma de genes.
No se encontraron valores de clasificación duplicados en IDO2.rnk. Por tanto, si lo desea puede realizar el GSEA para esta firma de genes.
No se encontraron valores de clasifi

## Corrección de archivos RNK con valores de clasificación duplicados

In [41]:
import pandas as pd
import os

def procesar_archivos_rnk():
    carpeta_origen = "Valores_duplicados"
    carpeta_destino = "Definitivas"

    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    for archivo_rnk in os.listdir(carpeta_origen):
        if archivo_rnk.endswith(".rnk"):
            rnk_path = os.path.join(carpeta_origen, archivo_rnk)
            procesar_archivo_rnk_individual(rnk_path, carpeta_destino, archivo_rnk)

def procesar_archivo_rnk_individual(rnk_path, carpeta_destino, nombre_archivo):
    nombre_archivo = os.path.splitext(os.path.basename(rnk_path))[0]
    xlsx_path = os.path.join(carpeta_destino, f"{nombre_archivo}_2.xlsx")

    df = pd.read_csv(rnk_path, delimiter='\t', header=None)
    df.to_excel(xlsx_path, index=False, header=None)

    while True:
        duplicados = df[df.duplicated(subset=[1], keep=False)]

        if duplicados.empty:
            break

        for i, fila in duplicados.iterrows():
            valor_original = fila[1]
            nueva_cifra_decimal = 10 ** -(len(str(valor_original).split('.')[1]) + 1)
            df.at[i, 1] = valor_original + nueva_cifra_decimal * (i + 1)

        df.to_excel(xlsx_path, index=False, header=None)

    nuevo_rnk_path = os.path.join(carpeta_destino, f"{nombre_archivo}_2.rnk")
    df.to_csv(nuevo_rnk_path, sep='\t', index=False, header=None)

    #respuesta_paso_7 = input(f"¿Desea conservar el archivo {nombre_archivo} y el libro de Excel generado? (Sí/No): ")
    #if respuesta_paso_7.lower() == 'sí' or respuesta_paso_7.lower() == 'si':
        #print(f"Se han conservado el archivo {nombre_archivo} y el libro de Excel creado")
        #return

    
    os.remove(xlsx_path)
        
    os.remove(rnk_path)
        
# Procesar todos los archivos .rnk en la carpeta "Definitivas"
procesar_archivos_rnk()

print("Corrección efectuada.")

Corrección efectuada.


## Conversión de archivos RNK a CSV
Para poder analizar las firmas de expresión génica escogidas con [blitzGSEA](https://doi.org/10.1093/bioinformatics/btac076) es necesario convertir los arhivos .rnk en archivos .csv (con valores separados por comas). Por ese motivo, el siguiente bloque de código lleva a cabo dicho proceso y guardas los CSVs resultantes en la carpeta indicada.

In [42]:
import os

def rnk_to_csv(input_folder, output_folder):
    # Verificar y crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos en la carpeta de entrada
    files = [f for f in os.listdir(input_folder) if f.endswith(".rnk")]

    for file in files:
        input_path = os.path.join(input_folder, file)
        output_path = os.path.join(output_folder, file.replace(".rnk", ".csv"))

        # Procesar el archivo .rnk y escribir en el archivo .csv
        with open(input_path, 'r') as input_file, open(output_path, 'w') as output_file:
            # Agregar encabezados
            output_file.write("0,1\n")

            for line in input_file:
                # Reemplazar tabulaciones por comas y escribir en el archivo de salida
                output_file.write(line.replace('\t', ','))

if __name__ == "__main__":
    # Rutas de las carpetas de entrada y salida
    input_folder = "Definitivas"
    output_folder = "CSV"

    # Llamar a la función para convertir archivos .rnk a .csv
    rnk_to_csv(input_folder, output_folder)
    
print("Conversión realizada.")

Conversión realizada.


## Herramientas de uso opcional:

### Eliminación de archivos RNK temporales

In [8]:
import os

def eliminar_archivos_por_extension(carpeta, extension):
    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(carpeta)

    # Filtrar los archivos por la extensión deseada y eliminarlos
    for archivo in archivos:
        if archivo.endswith(extension):
            ruta_archivo = os.path.join(carpeta, archivo)
            os.remove(ruta_archivo)

# Ejemplo de uso para eliminar archivos con extensión .rnk
carpeta_a_limpiar = 'Provisionales'
extension_deseada = '.rnk'

eliminar_archivos_por_extension(carpeta_a_limpiar, extension_deseada)

print("Eliminación concluida.")

### Reubicación de archivos RNK definitivos

#### Copiado de archivos RNK

In [7]:
import os
import shutil

def descargar_archivos_por_extension(carpeta_origen, carpeta_destino, extension):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Obtener la lista de archivos en la carpeta de origen
    archivos = os.listdir(carpeta_origen)

    # Filtrar los archivos por la extensión deseada
    archivos_filtrados = [archivo for archivo in archivos if archivo.endswith(extension)]

    # Iterar sobre los archivos filtrados y copiarlos a la carpeta de destino
    for archivo in archivos_filtrados:
        ruta_origen = os.path.join(carpeta_origen, archivo)
        ruta_destino = os.path.join(carpeta_destino, archivo)
        shutil.copy2(ruta_origen, ruta_destino)

# Ejemplo de uso para archivos con extensión .rnk
carpeta_origen = 'Definitivas'
carpeta_destino = 'C:/Users/marti/STRING-es/Firmas_genes'
extension_deseada = '.rnk'

descargar_archivos_por_extension(carpeta_origen, carpeta_destino, extension_deseada)

print("Proceso completado.")

#### Copiado y eliminación de archivos RNK

In [43]:
import os
import shutil

def descargar_y_eliminar_archivos(carpeta_origen, carpeta_destino, extension):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Obtener la lista de archivos en la carpeta de origen
    archivos = os.listdir(carpeta_origen)

    # Filtrar los archivos por la extensión deseada
    archivos_filtrados = [archivo for archivo in archivos if archivo.endswith(extension)]

    # Iterar sobre los archivos filtrados
    for archivo in archivos_filtrados:
        ruta_origen = os.path.join(carpeta_origen, archivo)
        ruta_destino = os.path.join(carpeta_destino, archivo)

        # Copiar el archivo a la carpeta de destino
        shutil.copy2(ruta_origen, ruta_destino)

        # Eliminar el archivo de la carpeta de origen
        os.remove(ruta_origen)

# Ejemplo de uso para archivos con extensión .rnk
carpeta_origen = 'Definitivas'
carpeta_destino = 'C:/Users/marti/STRING-es/Firmas_genes'
extension_deseada = '.rnk'

descargar_y_eliminar_archivos(carpeta_origen, carpeta_destino, extension_deseada)

print("Proceso completado.")

Proceso completado.


### Reubicación de archivos CSV

#### Copiado de archivos CSV

In [ ]:
import os
import shutil

def descargar_archivos_por_extension(carpeta_origen, carpeta_destino, extension):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Obtener la lista de archivos en la carpeta de origen
    archivos = os.listdir(carpeta_origen)

    # Filtrar los archivos por la extensión deseada
    archivos_filtrados = [archivo for archivo in archivos if archivo.endswith(extension)]

    # Iterar sobre los archivos filtrados y copiarlos a la carpeta de destino
    for archivo in archivos_filtrados:
        ruta_origen = os.path.join(carpeta_origen, archivo)
        ruta_destino = os.path.join(carpeta_destino, archivo)
        shutil.copy2(ruta_origen, ruta_destino)

# Ejemplo de uso para archivos con extensión .rnk
carpeta_origen = 'CSV'
carpeta_destino = 'C:/Users/marti/STRING-es/Firmas_genes'
extension_deseada = '.csv'

descargar_archivos_por_extension(carpeta_origen, carpeta_destino, extension_deseada)

print("Proceso completado.")

#### Copiado y borrado de archivos RNK

In [ ]:
import os
import shutil

def descargar_y_eliminar_archivos(carpeta_origen, carpeta_destino, extension):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Obtener la lista de archivos en la carpeta de origen
    archivos = os.listdir(carpeta_origen)

    # Filtrar los archivos por la extensión deseada
    archivos_filtrados = [archivo for archivo in archivos if archivo.endswith(extension)]

    # Iterar sobre los archivos filtrados
    for archivo in archivos_filtrados:
        ruta_origen = os.path.join(carpeta_origen, archivo)
        ruta_destino = os.path.join(carpeta_destino, archivo)

        # Copiar el archivo a la carpeta de destino
        shutil.copy2(ruta_origen, ruta_destino)

        # Eliminar el archivo de la carpeta de origen
        os.remove(ruta_origen)

# Ejemplo de uso para archivos con extensión .rnk
carpeta_origen = 'CSV'
carpeta_destino = 'C:/Users/marti/STRING-es/Firmas_genes'
extension_deseada = '.csv'

descargar_y_eliminar_archivos(carpeta_origen, carpeta_destino, extension_deseada)

print("Proceso completado.")

### Vaciado de las carpetas de archivos
Tras copiar los archivos que considere de interés a otra ubicación, puede ejecutar este código para borrar todos los archivos presentes en las carpetas indicadas en la "Lista de las carpetas".

In [44]:
import os

# Lista de las carpetas
carpetas = [
    "CSV",
    "Definitivas",
    "Listas_candidatos_SLoSDL",
    "Provisionales",
    "Valores_duplicados"
]

def eliminar_archivos(carpeta):
    # Listar todos los elementos en la carpeta
    for filename in os.listdir(carpeta):
        file_path = os.path.join(carpeta, filename)
        try:
            # Verificar si es un archivo y eliminarlo
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f'Archivo {file_path} eliminado')
            # Si es un directorio, lo ignoramos
        except Exception as e:
            print(f'No se pudo eliminar {file_path}. Razón: {e}')

# Eliminar archivos en cada carpeta
for carpeta in carpetas:
    eliminar_archivos(carpeta)

print("Proceso finalizado.")

Archivo Listas_candidatos_SLoSDL\CEGP_SLoSDL_resultados.xlsx eliminado
Archivo Provisionales\AFMID.rnk eliminado
Archivo Provisionales\ASPDH.rnk eliminado
Archivo Provisionales\HAAO.rnk eliminado
Archivo Provisionales\IDO1.rnk eliminado
Archivo Provisionales\IDO2.rnk eliminado
Archivo Provisionales\KMO.rnk eliminado
Archivo Provisionales\KYNU.rnk eliminado
Archivo Provisionales\NADSYN1.rnk eliminado
Archivo Provisionales\NAPRT.rnk eliminado
Archivo Provisionales\NAXD.rnk eliminado
Archivo Provisionales\NMNAT1.rnk eliminado
Archivo Provisionales\NMNAT2.rnk eliminado
Archivo Provisionales\NMNAT3.rnk eliminado
Archivo Provisionales\NMRK1.rnk eliminado
Archivo Provisionales\NMRK2.rnk eliminado
Archivo Provisionales\NNMT.rnk eliminado
Archivo Provisionales\NUDT12.rnk eliminado
Archivo Provisionales\PARP10.rnk eliminado
Archivo Provisionales\PARP16.rnk eliminado
Archivo Provisionales\PARP9.rnk eliminado
Archivo Provisionales\PTGIS.rnk eliminado
Archivo Provisionales\QPRT.rnk eliminado
Archiv